In [1]:
# ============================ Level-30 ============================
# Purged, embargoed time-series CV + probability threshold search + walk-forward backtest
# =================================================================

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Tuple, Optional, Iterable

import yfinance as yf
from datetime import datetime, timedelta

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# --------------------------- Config ---------------------------
TICKER        = "AAPL"
YEARS         = 5
FREQ          = "1d"        # "1h" / "1m" also works (consider more history)
VOL_SPAN      = 50
MIN_EVENTS    = 80          # how many labels we want minimally
BASE_H        = 10
UP_M, DN_M    = 1.0, 1.0

N_FOLDS       = 5           # Purged CV folds
EMBARGO_FRAC  = 0.01        # 1% embargo of the whole sample length
THRESH_GRID   = np.linspace(0.5, 0.9, 9)  # probability threshold candidates
TC_BP         = 5           # transaction costs in basis points per trade side (0.0001 = 1bp)
MAX_TURNOVER  = 0.5         # max daily change in position (to reduce churn)

RANDOM_STATE  = 42

# --------------------------- Helpers (robust price extraction etc.) ---------------------------
def ensure_series(x: pd.Series, name: Optional[str] = None) -> pd.Series:
    if isinstance(x, pd.DataFrame):
        x = x.iloc[:, 0]
    if not isinstance(x, pd.Series):
        raise TypeError("Expected a pandas Series.")
    if name:
        x = x.rename(name)
    x = pd.to_numeric(x, errors="coerce")
    x = x.dropna()
    x = x[~x.index.duplicated(keep="last")].sort_index()
    return x

def tz_to_naive_eastern(idx: pd.DatetimeIndex) -> pd.DatetimeIndex:
    if idx.tz is None:
        return idx
    return idx.tz_convert("US/Eastern").tz_localize(None)

def extract_close_series(df: pd.DataFrame) -> pd.Series:
    candidates = []
    if "Adj Close" in df.columns: candidates.append("Adj Close")
    if "Close" in df.columns:     candidates.append("Close")
    if not candidates:
        raise RuntimeError("No Close/Adj Close in data.")
    for col in candidates:
        s = df[col]
        if isinstance(s, pd.DataFrame):  # duplicate columns
            s = s.iloc[:, 0]
        s = ensure_series(s, "Close")
        if not s.empty:
            return s
    raise RuntimeError("Could not extract non-empty Close series.")

def load_prices(ticker: str, years: int, freq: str = "1d") -> pd.Series:
    start = (datetime.utcnow() - timedelta(days=int(365*years + 20))).date()
    df = yf.download(ticker, start=str(start), interval=freq, auto_adjust=True, progress=False)
    if df.empty:
        raise RuntimeError("No data returned. Check ticker/frequency/network.")
    if isinstance(df.index, pd.DatetimeIndex):
        idx = df.index
        try:
            idx = tz_to_naive_eastern(idx.tz_convert("UTC"))
        except Exception:
            if idx.tz is not None:
                idx = tz_to_naive_eastern(idx)
        df.index = idx
    return extract_close_series(df)

def ewma_vol(rets: pd.Series, span: int = 50) -> pd.Series:
    r = ensure_series(rets, "r")
    v = r.ewm(span=span, adjust=False).std()
    return v.rename("vol")

def cusum_filter(r: pd.Series, threshold: float) -> pd.DatetimeIndex:
    r = ensure_series(r, "r")
    s_pos = 0.0
    s_neg = 0.0
    t_events = []
    for t, x in r.items():
        xf = float(x)
        s_pos = max(0.0, s_pos + xf)
        s_neg = min(0.0, s_neg + xf)
        if s_pos > threshold:
            s_pos = 0.0
            t_events.append(t)
        elif s_neg < -threshold:
            s_neg = 0.0
            t_events.append(t)
    return pd.DatetimeIndex(t_events)

def get_vertical_barriers(t_events: pd.DatetimeIndex, H: int, full_index: pd.DatetimeIndex) -> pd.Series:
    full_index = pd.DatetimeIndex(full_index)
    pos_map = {ts: i for i, ts in enumerate(full_index)}
    n = len(full_index)
    out = {}
    for t0 in t_events:
        if t0 in pos_map:
            i = pos_map[t0]
        else:
            i = full_index.searchsorted(t0, side="left")
        j = min(n - 1, i + int(H))
        out[t0] = full_index[j]
    return pd.Series(out, name="t1")

def first_cross_idx(path_vals: np.ndarray, level: float, cmp: str) -> Optional[int]:
    if cmp == "ge":
        mask = path_vals >= level
    elif cmp == "le":
        mask = path_vals <= level
    else:
        raise ValueError("cmp must be 'ge' or 'le'")
    where = np.where(mask)[0]
    return int(where[0]) if where.size else None

def get_triple_barrier_labels(
    close: pd.Series,
    t_events: pd.DatetimeIndex,
    vbar: pd.Series,
    up_m: float,
    dn_m: float,
    daily_vol: pd.Series
) -> pd.DataFrame:
    close = ensure_series(close, "Close")
    daily_vol = ensure_series(daily_vol, "vol")
    vbar = vbar.dropna()
    rows = []
    for t0 in t_events:
        if t0 not in close.index or t0 not in vbar.index:
            continue
        t1 = vbar.loc[t0]
        if t1 not in close.index:
            j = close.index.searchsorted(t1, side="left")
            j = max(0, min(j, len(close) - 1))
            t1 = close.index[j]
        c0 = float(close.loc[t0])
        trgt = float(daily_vol.get(t0, np.nan))
        if not np.isfinite(trgt) or trgt <= 0:
            continue
        up_lvl = c0 * (1 + up_m * trgt)
        dn_lvl = c0 * (1 - dn_m * trgt)

        path_idx = close.loc[t0:t1].index
        path = close.loc[path_idx].to_numpy(dtype=float)

        iu = first_cross_idx(path, up_lvl, "ge")
        idn = first_cross_idx(path, dn_lvl, "le")

        label = 0
        t_hit = t1
        if iu is not None and idn is not None:
            if iu < idn:
                label = 1;  t_hit = path_idx[iu]
            elif idn < iu:
                label = -1; t_hit = path_idx[idn]
            else:
                label = 0;  t_hit = path_idx[iu]
        elif iu is not None:
            label = 1;  t_hit = path_idx[iu]
        elif idn is not None:
            label = -1; t_hit = path_idx[idn]
        else:
            rvt = float(close.loc[t1] / c0 - 1.0)
            label = 1 if rvt > 0 else (-1 if rvt < 0 else 0)
            t_hit = t1

        rows.append({"t0": t0, "t1": t1, "t_hit": t_hit, "label": label, "trgt": trgt})

    out = pd.DataFrame.from_records(rows).set_index("t0").sort_index()
    return out

def build_features(px: pd.Series) -> pd.DataFrame:
    px = ensure_series(px, "Close")
    r1 = px.pct_change().fillna(0.0)
    r5 = px.pct_change(5).fillna(0.0)
    vol20 = r1.rolling(20).std().fillna(0.0)
    mom10 = px.pct_change(10).fillna(0.0)
    ma10 = px.rolling(10).mean()
    ma20 = px.rolling(20).mean()
    ma_ratio = (ma10 / ma20 - 1.0).replace([np.inf, -np.inf], 0.0).fillna(0.0)
    feat = pd.DataFrame({"r1": r1, "r5": r5, "vol20": vol20, "mom10": mom10, "ma_ratio": ma_ratio}, index=px.index)
    return feat

def adaptive_events_and_labels(
    close: pd.Series,
    rets: pd.Series,
    vol_span: int = 50,
    min_events: int = 80,
    base_H: int = 10,
    up_m: float = 1.0,
    dn_m: float = 1.0
) -> Tuple[pd.DataFrame, pd.Series, pd.DatetimeIndex, int, float]:
    close = ensure_series(close, "Close")
    rets  = ensure_series(rets, "r")
    daily_vol = ewma_vol(rets, span=vol_span).clip(lower=1e-8).fillna(0.0)

    vol_std = float(rets.std()) if np.isfinite(float(rets.std())) else 0.01
    thr_fracs = [0.50, 0.35, 0.25, 0.18, 0.12, 0.08, 0.05, 0.03]
    H_choices = [base_H, int(base_H*1.5), base_H*2, base_H*3]

    labels = pd.DataFrame()
    used_thr, used_H, events_idx = None, None, pd.DatetimeIndex([])

    for H_try in H_choices:
        for frac in thr_fracs:
            thr = max(1e-4, frac * vol_std)
            events_idx = cusum_filter(rets, thr)
            if len(events_idx) == 0:
                continue
            vbar = get_vertical_barriers(events_idx, H_try, close.index)
            lab = get_triple_barrier_labels(close, events_idx, vbar, up_m, dn_m, daily_vol).dropna()
            if len(lab) >= min_events:
                labels = lab
                used_thr, used_H = thr, H_try
                break
        if len(labels) >= min_events:
            break

    if labels.empty:
        for H_try in H_choices[max(1, len(H_choices)//2):]:
            for frac in thr_fracs + [0.02, 0.015, 0.01]:
                thr = max(5e-5, frac * vol_std)
                events_idx = cusum_filter(rets, thr)
                if len(events_idx) == 0:
                    continue
                vbar = get_vertical_barriers(events_idx, H_try, close.index)
                lab = get_triple_barrier_labels(close, events_idx, vbar, up_m*0.5, dn_m*0.5, daily_vol).dropna()
                if len(lab) >= max(10, min_events//2):
                    labels = lab
                    used_thr, used_H = thr, H_try
                    break
            if not labels.empty:
                break

    return labels, daily_vol, events_idx, (used_H or base_H), float(used_thr or 0.0)

# --------------------------- Purged, embargoed CV ---------------------------
@dataclass
class PurgedFold:
    tr_idx: np.ndarray
    te_idx: np.ndarray

def make_purged_folds(index: pd.DatetimeIndex,
                      t1: pd.Series,
                      n_folds: int,
                      embargo_frac: float) -> List[PurgedFold]:
    """
    Split the time index into ordered folds; purge train samples whose (t0,t1) overlap test,
    and embargo a gap after each test fold.
    """
    n = len(index)
    fold_sizes = np.full(n_folds, n // n_folds, dtype=int)
    fold_sizes[: n % n_folds] += 1
    bounds = np.cumsum(fold_sizes)

    folds = []
    start = 0
    embargo = int(np.ceil(n * embargo_frac))

    for b in bounds:
        te_slice = np.arange(start, b)
        te_idx = te_slice

        te_times = index[te_idx]
        te_start, te_end = te_times[0], te_times[-1]

        # Purge: remove any train event whose span intersects [te_start, te_end]
        tr_mask = np.ones(n, dtype=bool)
        if not t1.empty:
            # events exist at subset of index; we use conservative rule:
            # any sample whose timestamp is within [te_start, te_end] gets removed from train
            tr_mask[(index >= te_start) & (index <= te_end)] = False

        # Embargo: remove the next embargo samples after test block
        emb_start = min(n, b)
        emb_end = min(n, b + embargo)
        tr_mask[emb_start:emb_end] = False

        # Also remove the test indices from train
        tr_mask[te_idx] = False

        tr_idx = np.nonzero(tr_mask)[0]
        folds.append(PurgedFold(tr_idx=tr_idx, te_idx=te_idx))

        start = b
    return folds

# --------------------------- Backtest helpers ---------------------------
def sharpe_ratio(rets: pd.Series) -> float:
    if len(rets) < 2: return 0.0
    mu = rets.mean()
    sd = rets.std(ddof=0)
    return 0.0 if sd == 0 else float(np.sqrt(252) * mu / sd)

def apply_tc(positions: pd.Series, tc_bp: float) -> pd.Series:
    """
    Subtract transaction costs when position changes sign/size.
    positions: target in [0,1] (long/flat).
    TC applied on abs(delta_pos) * tc_rate.
    """
    tc_rate = tc_bp / 10000.0
    dpos = positions.diff().fillna(positions.iloc[0])
    return dpos.abs() * tc_rate

def cap_turnover(positions: pd.Series, max_turnover: float) -> pd.Series:
    """
    Limit per-step change in position to max_turnover.
    """
    pos = positions.copy()
    for i in range(1, len(pos)):
        delta = pos.iloc[i] - pos.iloc[i-1]
        if abs(delta) > max_turnover:
            pos.iloc[i] = pos.iloc[i-1] + np.sign(delta) * max_turnover
    return pos.clip(0.0, 1.0)

# ============================ Main Flow ============================
px = load_prices(TICKER, YEARS, FREQ)
rets = px.pct_change().replace([np.inf, -np.inf], 0.0).fillna(0.0)

labels, daily_vol, events_idx, H_used, thr_used = adaptive_events_and_labels(
    close=px, rets=rets,
    vol_span=VOL_SPAN, min_events=MIN_EVENTS,
    base_H=BASE_H, up_m=UP_M, dn_m=DN_M
)
print(f"[Adaptive] events={len(events_idx)} labels={len(labels)}  H={H_used}  thr≈{thr_used:.6g}")
if labels.empty:
    raise RuntimeError("No labels. Increase YEARS or lower MIN_EVENTS / switch to higher frequency.")

# Features & target aligned on t0
X_all = build_features(px)
X = X_all.reindex(labels.index).dropna()
y = (labels["label"] == 1).astype(int).reindex(X.index)
t1 = labels["t1"].reindex(X.index)  # for purging logic; here we use simple conservative rule on timestamps

# Common scaler & model
scaler = StandardScaler()
clf = RandomForestClassifier(
    n_estimators=400, max_depth=7,
    random_state=RANDOM_STATE, n_jobs=-1,
    class_weight="balanced_subsample"
)

# Build folds on the feature index (time-ordered)
folds = make_purged_folds(X.index, t1, n_folds=N_FOLDS, embargo_frac=EMBARGO_FRAC)

val_scores = []
fold_reports = []
per_fold_best = []

for k, fold in enumerate(folds, 1):
    tr_idx, te_idx = fold.tr_idx, fold.te_idx
    X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
    y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

    if y_tr.nunique() < 2 or len(y_te) == 0:
        continue

    X_tr_sc = pd.DataFrame(scaler.fit_transform(X_tr), index=X_tr.index, columns=X_tr.columns)
    X_te_sc = pd.DataFrame(scaler.transform(X_te), index=X_te.index, columns=X_te.columns)

    clf.fit(X_tr_sc, y_tr)
    proba = pd.Series(clf.predict_proba(X_te_sc)[:, 1], index=X_te_sc.index)

    # Threshold search to maximize Sharpe on validation (using realized next-day returns)
    # Strategy: position_t = 1 if proba_t >= thr else 0; pnl_t+1 = position_t * rets_{t+1}
    # Align: use returns on the same universe dates:
    r = rets.reindex(proba.index.union(rets.index)).reindex(proba.index).shift(-1).fillna(0.0)

    best_thr, best_sh = 0.5, -np.inf
    for thr in THRESH_GRID:
        pos = (proba >= thr).astype(float)
        pos = cap_turnover(pos, MAX_TURNOVER)
        tc = apply_tc(pos, TC_BP)
        pnl = pos * r - tc
        sh = sharpe_ratio(pnl)
        if sh > best_sh:
            best_sh, best_thr = sh, thr

    per_fold_best.append((best_thr, best_sh))
    val_scores.append(best_sh)

    y_pred = (proba >= best_thr).astype(int)
    rep = classification_report(y_te, y_pred, digits=3, output_dict=True, zero_division=0)
    fold_reports.append((k, rep))

mean_val_sharpe = float(np.mean(val_scores)) if val_scores else 0.0
best_thr_overall = float(np.median([t for t, _ in per_fold_best])) if per_fold_best else 0.6

print(f"\nCV mean Sharpe (validation): {mean_val_sharpe:.3f}")
print(f"Chosen probability threshold (median of per-fold best): {best_thr_overall:.2f}")

# --------------------------- Refit on full sample & Walk-forward backtest ---------------------------
X_sc_all = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
clf.fit(X_sc_all, y)

proba_all = pd.Series(clf.predict_proba(X_sc_all)[:, 1], index=X_sc_all.index)
pos_raw = (proba_all >= best_thr_overall).astype(float)
pos = cap_turnover(pos_raw, MAX_TURNOVER)

# Backtest on the same dates (out-of-sample equivalent is complex; here we show realized pnl using same-aligned returns)
r_aligned = rets.reindex(pos.index).shift(-1).fillna(0.0)
tc = apply_tc(pos, TC_BP)
pnl = pos * r_aligned - tc

eq = (1 + pnl).cumprod()
sh = sharpe_ratio(pnl)
cagr = (eq.iloc[-1]) ** (252/len(eq)) - 1 if len(eq) > 10 else np.nan
dd = (eq / eq.cummax() - 1.0).min()
hit_rate = (pnl > 0).mean()

print("\n=== Walk-forward Backtest (long/flat meta-signal) ===")
print(f"Sharpe: {sh:.3f}  CAGR: {cagr:.2%}  MaxDD: {dd:.2%}  Hit%: {hit_rate:.2%}")
print(f"Avg daily turnover: {pos.diff().abs().mean():.3f}  (Max cap {MAX_TURNOVER})")
print(f"Avg daily TC paid (bp): {tc.mean()*10000:.2f}")

# Save artifacts
out_cv = pd.DataFrame({
    "fold": [i for i,_ in fold_reports],
    "val_sharpe": [s for _, s in per_fold_best],
    "best_thr":   [t for t, _ in per_fold_best]
})
out_cv.to_csv(f"{TICKER}_level30_cv_summary.csv", index=False)

bt = pd.DataFrame({
    "proba": proba_all,
    "pos_raw": pos_raw,
    "pos_capped": pos,
    "rets_next": r_aligned,
    "tc": tc,
    "pnl": pnl,
    "equity": eq
})
bt.to_csv(f"{TICKER}_level30_backtest.csv", index_label="date")

print(f"\nSaved: {TICKER}_level30_cv_summary.csv, {TICKER}_level30_backtest.csv")


C:\Users\adity\AppData\Local\Temp\ipykernel_12488\2997544063.py:72: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start = (datetime.utcnow() - timedelta(days=int(365*years + 20))).date()


[Adaptive] events=708 labels=708  H=10  thr≈0.00895125

CV mean Sharpe (validation): 2.240
Chosen probability threshold (median of per-fold best): 0.55

=== Walk-forward Backtest (long/flat meta-signal) ===
Sharpe: 6.686  CAGR: 279.59%  MaxDD: -5.65%  Hit%: 44.35%
Avg daily turnover: 0.238  (Max cap 0.5)
Avg daily TC paid (bp): 1.19

Saved: AAPL_level30_cv_summary.csv, AAPL_level30_backtest.csv
